# Imports


In [151]:
import pandas as pd
from sodapy import Socrata
import sqlite3
from sqlite3 import Error


# Create my database
 

In [170]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('sqlite3 version:', sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            print('connected to', db_file)
            return conn
        else:
            print("Failed")
            


conn = create_connection('database/rlc.db')
c = conn.cursor()
#conn.close()

sqlite3 version: 2.6.0
connected to database/rlc.db


## create sodapy client

In [171]:
## referenced example from https://github.com/xmunoz/sodapy/issues/52

client = Socrata("data.cityofchicago.org", None)  # starts up my client object
data = client.get("spqx-js37", #speed cams are at 'hhkd-xvj4' 
                     #where='violation_date BETWEEN \'2015-01-01T00:00:00.000\' AND \'2020-12-20T00:00:00.000\'',
                     limit=100,
                    )


## Clunky, but I found easiest to go to df, then to db

In [172]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(data)
results_df.head()

,intersection,camera_id,address,violation_date,violations,x_coordinate,y_coordinate,latitude,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu
0,111TH AND HALSTED,2424,800 W 111TH ST,2020-12-16T00:00:00.000,3,1172945.404219168,1831282.917027882,41.692465363,-87.642440901,"{'latitude': '41.692465363421874', 'longitude'...",45,21861,74,315,22
1,111TH AND HALSTED,2422,11100 S HALSTED ST,2020-12-16T00:00:00.000,13,1172923.657689312,1831209.044179248,41.692263123,-87.642522688,"{'latitude': '41.692263123425136', 'longitude'...",45,21861,74,663,22
2,119TH AND HALSTED,2404,800 W 119TH STREET,2020-12-16T00:00:00.000,8,1173111.174088158,1825985.001014405,41.677923389,-87.641989646,"{'latitude': '41.677923389134385', 'longitude'...",45,21861,50,642,22
3,119TH AND HALSTED,2402,11900 S HALSTED,2020-12-16T00:00:00.000,14,1173094.774616191,1825911.054609338,41.677720829,-87.642051845,"{'latitude': '41.67772082924762', 'longitude':...",45,21861,50,662,22
4,31ST ST AND MARTIN LUTHER KING DRIVE,2121,3100 S DR MARTIN L KING,2020-12-16T00:00:00.000,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
# pare down the df to make suitable as db.  Remember only  allowable types TEXT.NUMERIC.INTEGER.REAL.BLOB.
valid_cols = results_df.columns[:10]
results_df = results_df[valid_cols]

# coerce my data types
results_df['camera_id'] = results_df['camera_id'].apply(int)
results_df['x_coordinate'] = results_df['x_coordinate'].apply(float)
results_df['y_coordinate'] = results_df['y_coordinate'].apply(float)
results_df['latitude'] = results_df['latitude'].apply(float)
results_df['longitude'] = results_df['longitude'].apply(float)
results_df['location'] = results_df['location'].apply(str)


results_df.head()
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
intersection      100 non-null object
camera_id         100 non-null int64
address           100 non-null object
violation_date    100 non-null object
violations        100 non-null object
x_coordinate      98 non-null float64
y_coordinate      98 non-null float64
latitude          98 non-null float64
longitude         98 non-null float64
location          100 non-null object
dtypes: float64(4), int64(1), object(5)
memory usage: 7.9+ KB


## create a table if it doesn't exist

In [229]:
d_types = ['text',
         'integer',
         'text',
         'text',
         'integer',
         'real',
         'real',
         'real',
         'real',
         'text']



#create_text = 'CREATE TABLE VIOLATIONS ({}, {}, {}, {}, {}, {}, {}, {}, {} ,{})'.format(*insert_values)

## write data from query to sqlite db

In [238]:
# def create_table(c, mytable, cols):
#     print('anything')
#     try:
#         cols_str = "("
#         print(cols_str)
        
#         for col in cols: 
#             cols_str += col + ','
#         else:
#             cols_str += ')'
        
#         print(cols_str)
#         print("CREATE TABLE {} {}".format(mytable, cols)
#         #c.execute("CREATE TABLE {} {}".format(mytable, cols)
#         #conn.commit()
#     except:
#         print('Failed to add', mytable)
#         pass

def create_table(c, mytable, cols, d_type):
    
    # make my columns into sql string compatible
    cols_str = "("
    for i in range(len(cols)):
        cols_str += cols[i] + ' ' + d_type[i] + ', '
    else:
        cols_str = cols_str[:-2] + ')'
    
    my_sql = "CREATE TABLE {} {}".format(mytable, cols_str)
    print(my_sql) 
    
    try:     
        c.execute(my_sql)           
    except Exception as e:
        print('fail', e)
        

def insert_violation(c, col_names, new_data):
    """
    Create a new task
    :param conn:
    :param new_data:
    :return:
    """
    
    vals = ('('+'?,'*len(new_data))[:-1]+')'
    cols = str(tuple([x for x in col_names])).replace(' ', '').replace("'", "").replace(',', ', ')
    sql = '''INSERT INTO violations{}VALUES{}'''.format(cols, vals)
    # example from sql site 
    # sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)VALUES(?,?,?,?,?,?) '''
    
    #print(sql)
    #print(tuple(new_data))
    c.execute(sql, new_data)

    return c.lastrowid


# only do this once
#create_table(c, 'violations', results_df.columns, d_types)
#conn.commit()

insert_violation(c, results_df.columns, results_df.iloc[1, :])
conn.commit()


#c.execute('CREATE TABLE CARS (Brand text, Price number)')   # this is example from sqlite website
#c.execute(create_text)
#conn.commit()


In [239]:
def sql_fetch(c):
    c.execute('SELECT name from sqlite_master where type= "table"')
    print(c.fetchall())

sql_fetch(c)

[('violations',)]


## Now let's see if our write to db worked as expected

In [241]:
c.execute("SELECT intersection FROM violations;")
print(c.fetchall())

[('111TH AND HALSTED',), ('111TH AND HALSTED',)]


In [242]:
conn.close()

In [243]:
conn